In [1]:
import requests
from bs4 import BeautifulSoup
from random import choice
import pandas as pd

In [2]:
def get_proxy():
    url = "https://www.sslproxies.org/"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    return {'https': choice(list(map(lambda x:x[0] + ':' + x[1], list(zip(map(lambda x: x.text, soup.findAll('td')[::8]), map(lambda x: x.text, soup.findAll('td')[1::8]))))))}

In [3]:
def proxy_request(url, **kwargs):
    while True:
        try:
            proxy = get_proxy()
            r = requests.get(url, proxies=proxy, timeout=5, **kwargs)
            if len(r.text) < 200:
                continue
            break
        except:
            pass
    return r

In [4]:
def get_info(url):
    
    r = proxy_request(url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    address = soup.find(class_="tab-pane").text.split('Check on Google Maps')[0].replace("\n", "").strip()
    phone = soup.find(class_="tab-pane").text.split('Check on Google Maps')[1].replace("\n", "").strip()
    coordinates = soup.find('span', text = 'Check on Google Maps').parent.get('href').split('=')[1]
    
    return [address, phone, coordinates]
    

In [ ]:
df = pd.read_excel("C:\Dataframes\Regal Cinemas.xlsx")

for i in range(500, 557):
    try:
        info = get_info(df['Link'][i])
        df['Address'][i] = info[0]
        df['Phone'][i] = info[1]
        df['Lat,Lng'][i] = info[2]
        print('Current progress: ' + str(i))
    except:
        print("Error at " + str(i))
        continue

df.to_excel("C:\Dataframes\Regal Cinemas.xlsx", index=False)